In [1]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers,optimizers,losses

from json import load

In [2]:
with open("../data/example_build.json","r") as build_file:
    build_config = load(build_file)
    
with open("../data/layer_config.json","r") as config_file:
    layer_config = load(config_file)

In [3]:
inputs = []

for _id,config in build_config.items():
    if config['type'] == 'Input':
        inputs.append(_id)
        
build_config['input_nodes'] = inputs

levels = [ set() for i in range(len(build_config))]
def setLevel(node,config,di=0):
    levels[di].add(node)
    if build_config[node]['connections']['outbound']:
        for next_node in build_config[node]['connections']['outbound']: 
            setLevel(next_node,build_config,di+1)


for inp in inputs:
    setLevel(inp,build_config,0)
    
build_config['levels'] = levels
    
levels = [list(level) for level in levels if len(level)]
levels

[['input_1'],
 ['conv2d_1'],
 ['conv2d_2'],
 ['globalaveragepooling2d_1'],
 ['dense_1'],
 ['model_1'],
 ['compile_1'],
 ['train_1']]

In [4]:
def build_inbound(inbound:list)->str:
    inbound = [ f"""build_config['train']['{node}']""" for node in inbound ]
    return ( "([" + ", ".join(inbound) + "])" if len(inbound) > 1 else "(" + inbound[0] + ")" ) if len(inbound) else ""

def set_argument(argument,config):
    value = config['value']
    value = ( None if value == 'None' else f"'{value}'" ) if config['type'] == 'text' else value
    return f"    {argument}={value},"
    
def build_arguments(arguments:dict)->str:
    arguments = '\n'.join([ set_argument(arg,cnf) for arg,cnf in arguments.items() ])
    return arguments

def build_input(layer,build_config,*args,**kwargs)->str:
    arguments =  build_arguments(layer['arguments'])
    return eval(f"""layers.Input(
{arguments}
)
""")

def build_dense(layer,build_config,*args,**kwargs)->str:
    arguments =  build_arguments(layer['arguments'])
    inbound = build_inbound(layer['connections']['inbound'])
    
    return eval(f"""layers.{layer['type']}(
{arguments}
){inbound}
""")

def build_conv2d(layer,build_config,*args,**kwargs)->str:
    arguments =  build_arguments(layer['arguments'])
    inbound = build_inbound(layer['connections']['inbound'])
    
    return eval(f"""layers.{layer['type']}(
{arguments}
){inbound}
""")

def build_globalaverage2d(layer,build_config,*args,**kwargs)->str:
    inbound = layer['connections']['inbound']
    inbound = build_inbound(layer['connections']['inbound'])
    
    return eval(f"""layers.{layer['type']}(){inbound}""")

def build_model(layer,build_config,*args,**kwargs)->str:
    in_nodes = [ f"""build_config['train']['{node}']""" for node in build_config['input_nodes']]
    out_nodes = [ f"""build_config['train']['{node}']""" for node in layer['connections']['inbound']]
    return eval(f"""keras.Model(
    [ {', '.join(in_nodes)}, ],
    [ {', '.join(out_nodes)}, ]
)""")

def build_compile(layer,build_config,*args,**kwargs)->str:
    model, = layer['connections']['inbound']
    eval(f"""
build_config['train']['{model}'].compile(
    optimizer='{layer['arguments']['optmizer']['value']}',
    loss='{layer['arguments']['loss']['value']}'
)""")
    return True

def build_train(layer,build_config,*args,**kwargs)->str:
    return ""

build_functions = {
    "Input":build_input,
    "Conv2D":build_conv2d,
    "GlobalAveragePooling2D":build_globalaverage2d,
    "Dense":build_dense,
    "Model":build_model,
    "Compile":build_compile,
    "Train":build_train
}

build_config['train'] = {
    
}

for level in levels:
    for layer in level:
        layer = build_config[layer]
        layer_build = build_functions[layer['type']](layer,build_config) 
        build_config['train'][layer['id']] = layer_build
        
build_config['train']

{'input_1': <KerasTensor: shape=(None, 28, 28, 1) dtype=float32 (created by layer 'input_1')>,
 'conv2d_1': <KerasTensor: shape=(None, 14, 14, 32) dtype=float32 (created by layer 'conv2d')>,
 'conv2d_2': <KerasTensor: shape=(None, 7, 7, 32) dtype=float32 (created by layer 'conv2d_1')>,
 'globalaveragepooling2d_1': <KerasTensor: shape=(None, 32) dtype=float32 (created by layer 'global_average_pooling2d')>,
 'dense_1': <KerasTensor: shape=(None, 10) dtype=float32 (created by layer 'dense')>,
 'model_1': <tensorflow.python.keras.engine.functional.Functional at 0x1f286889790>,
 'compile_1': True,
 'train_1': ''}

In [5]:
model = build_config['train']['model_1']

In [7]:
(X,Y,),(x,y) = tf.keras.datasets.mnist.load_data()

X = X.reshape(-1,28,28,1) / 255
x = x.reshape(-1,28,28,1) / 255

Y = keras.utils.to_categorical(Y)
y = keras.utils.to_categorical(y)

In [13]:
class TfGui(keras.callbacks.Callback):
    batch = None
    logs = []
    def on_batch_end(self,batch,logs=None):
        self.batch = batch
        self.logs.append(logs)
    
    def on_epoch_end(self,epoch,logs=None):
        print (self.model)
    
tf_gui = TfGui()

In [14]:
model.fit(x=X,y=Y,epochs=1,batch_size=32,validation_data=(x,y),callbacks=[tf_gui])

1875/1875 [==============================] - 7s 4ms/step - loss: 1.3918 - val_loss: 0.9976


In [20]:
dir(tf_gui.model)

['_TF_MODULE_IGNORED_PROPERTIES',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_activity_regularizer',
 '_add_trackable',
 '_add_variable_with_custom_getter',
 '_assert_compile_was_called',
 '_assert_weights_created',
 '_auto_track_sub_layers',
 '_autocast',
 '_autographed_call',
 '_base_model_initialized',
 '_build_input_shape',
 '_call_accepts_kwargs',
 '_call_arg_was_passed',
 '_call_fn_arg_defaults',
 '_call_fn_arg_positions',
 '_call_fn_args',
 '_call_full_argspec',
 '_callable_losses',
 '_cast_single_input',
 '_check_call_args',
 '_checkpoint_dependencies',
 '_clear_losses',
 '_compile_was_called',
 '_compiled_tr